<a href="https://colab.research.google.com/github/jbcgames/Lab_1_Controls/blob/main/Resumen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identificación de Modelo FOPDT mediante el Método de Smith

###Introducción

En el marco del laboratorio TempControlLab UdeA, implementamos el método de Smith para identificar los parámetros de un modelo FOPDT (First Order Plus Dead Time) que describe la dinámica térmica del sistema. Utilizamos datos experimentales obtenidos al aplicar un escalón de potencia al calentador Heater1 y registramos la respuesta de temperatura con el sensor LM35. El proceso incluyó filtrado de señales, identificación gráfica de parámetros y validación del modelo. A continuación, detallamos cada etapa del análisis.

#1. Procesamiento de Datos

Objetivo: Preparar los datos experimentales para el análisis, eliminando ruido y enfocándonos en la fase relevante del calentamiento.

Código y Explicación:

In [ ]:
# 1. Leer datos desde GitHub
url_datos = 'https://raw.githubusercontent.com/jbcgames/Lab_1_Controls/main/Datos.txt'
df = pd.read_csv(url_datos)

# 2. Filtrar datos hasta t = 450 s (excluir enfriamiento)
mask = (tiempo <= 450)
tiempo = tiempo[mask]
T1 = T1[mask]
pwm = pwm[mask]

Justificación: Limitamos el análisis a los primeros 450 segundos para evitar incluir la fase de enfriamiento, que no es relevante para el modelo FOPDT en lazo abierto.

#2. Detección del Escalón y Filtrado de Señal

Objetivo: Identificar el instante exacto del escalón de potencia y suavizar la señal de temperatura para reducir el ruido.

Código y Explicación:

In [ ]:
# 3. Detectar el escalón (PWM: 0% → 100%)
indices_escalon = np.where((pwm[:-1] == 0) & (pwm[1:] > 0))[0]
t_escalon = tiempo[indices_escalon[0] + 1]
print(f"Escalón detectado en t = {t_escalon:.2f} s")

# 4. Aplicar filtro Savitzky-Golay
T1_smooth = savgol_filter(T1, window_length=31, polyorder=3)

Resultado: El escalón se detectó en t=10.05s.

Filtrado: El filtro Savitzky-Golay preservó la tendencia principal mientras eliminaba ruido.

#3. Cálculo de Parámetros FOPDT

Objetivo: Estimar la ganancia (K), constante de tiempo (τ) y tiempo muerto (L) mediante el método de Smith.

Código y Explicación:

In [ ]:
# 5. Calcular puntos al 28.3% y 63.2% del cambio
y_28 = y0 + 0.283 * delta_y
y_63 = y0 + 0.632 * delta_y
t_28 = tiempo[np.argmin(np.abs(T1_smooth - y_28))]
t_63 = tiempo[np.argmin(np.abs(T1_smooth - y_63))]

# 6. Calcular parámetros
tau = 1.5 * (t_63 - t_28)  # Constante de tiempo
L = t_63 - tau              # Tiempo muerto
K = delta_y / pwm[idx_escalon + 1]  # Ganancia

Resultados:

Ganancia (K):
0.361 °C


Constante de tiempo (τ):
135.26 s


Tiempo muerto (L):
25.07 s


#4. Simulación y Validación del Modelo

Objetivo: Comparar los datos reales con la respuesta del modelo FOPDT estimado.

Código y Explicación:

In [ ]:
# 7. Simular modelo FOPDT
respuesta_modelo = np.where(t_modelo < L, y0, y0 + K * pwm[idx_escalon+1] * (1 - np.exp(-(t_modelo - L)/tau)))

# 8. Graficar resultados
plt.plot(tiempo, T1, label='Datos originales', alpha=0.7)
plt.plot(tiempo, T1_smooth, label='Datos suavizados')
plt.plot(t_modelo, respuesta_modelo, '--', label='Modelo FOPDT')
plt.axvline(L, color='gray', linestyle='--', label='Tiempo muerto L')

#Gráfico Resultante:

Identificación con Método de Smith

Observaciones: El modelo ajusta bien en estado estacionario, pero hay discrepancias menores en la fase transitoria, posiblemente debido a no linealidades no capturadas.

#5. Conclusiones

Efectividad del método: El método de Smith, combinado con el filtrado de datos, proporcionó parámetros coherentes con la dinámica térmica del sistema.

Mejoras implementadas:

-Exclusión de la fase de enfriamiento (t>450 s).

-Filtrado de ruido con Savitzky-Golay para una identificación más precisa.


